In [7]:
import pandas as pd
import json
import numpy as np
import re

In [19]:
info = pd.read_excel('SymMap v2.0, SMHB file.xlsx')
info

,Herb_id,Chinese_name,Pinyin_name,Latin_name,English_name,Properties_Chinese,Properties_English,Meridians_Chinese,Meridians_English,Class_Chinese,Class_English,UsePart,TCMID_id,TCM-ID_id,TCMSP_id,Link_herb_id,Alias,HERBDB_ID,Suppress
0,1,矮地茶,Aidicha,"Ardisiae Japonicae Herba,Ardisiae Japonicae",Japanese Ardisia Herb,"辛,微苦,平","Pungent,Slightly Bitter,Calm","肺,肝","Lung,Liver",止咳平喘药,Antitussive Antiasthmetics,NaN,5072.0,2.0,1.0,NaN,NaN,HERB000036,0
1,2,艾叶,Aiye,"Folium Artemisiae Argyi,Artemisiae Argyi Folium",leaf of Argy Wormwood,"辛,苦,温,有小毒","Pungent,Bitter,Warm,Slightly Toxic","肝,脾,肾","Liver,Spleen,Kidney",温经止血药,Channel-Warming Hemostatic,leaf,6336.0,3.0,2.0,NaN,NaN,HERB000066,0
2,3,安息香,Anxixiang,Benzoinum,Benzoin,"辛,苦,平","Pungent,Bitter,Calm","心,脾","Heart,Spleen",开窍药,Resuscitative Stimulant; Resuscitative Medicinal,balsam,8019.0,6.0,3.0,NaN,NaN,HERB000090,0
3,4,八角茴香,Bajiaohuixiang,"Fructus Anisi Stellati,Anisi Stellati Fructus",Chinese Star Anise,"辛,温","Pungent,Warm","肝,肾,脾,胃","Liver,Kidney,Spleen,Stomach",温里药,Warming Interior Drugs,NaN,6861.0,10.0,4.0,NaN,NaN,HERB000137,0
4,5,八角莲,Bajiaolian,NaN,Dysosmae Verspiellis Rhixoma Et Radix,"苦,辛,温,有毒","Bitter,Pungent,Warm,Toxic","肺,肝","Lung,Liver",清热解毒药,Antipyretic Detoxicate Drugs,NaN,NaN,NaN,5.0,NaN,NaN,HERB000139,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
698,699,翼首草,Yishoucao,Pterocephali Herba,NaN,"苦,寒,小毒","Bitter,Cold,Slightly Toxic",NaN,NaN,解毒杀虫燥湿止痒药,Medicinal For Detoxification、Parasiticide、Dryi...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
699,700,蟾酥,Chansu,Bufonis Venenum,NaN,"辛,温,有毒","Pungent,Warm,Toxic",心,Heart,攻毒杀虫止痒药,Toxin-Attacking And Worm-Expelling And Itch-Re...,NaN,NaN,NaN,NaN,NaN,NaN,HERB000650,0
700,701,鳖甲,Biejia,Trionycis Carapax,NaN,"咸,微寒","Salty,Slightly Cold","肝,肾","Liver,Kidney",补虚药,Tonifying And Replenishing Medicinal,NaN,NaN,NaN,NaN,NaN,NaN,HERB000484,0
701,702,麝香,Shexiang,Moschus,NaN,"辛,温","Pungent,Warm","心,脾","Heart,Spleen",开窍药,Resuscitative Stimulant; Resuscitative Medicinal,NaN,NaN,NaN,NaN,NaN,NaN,HERB004984,0


In [21]:
chinese_names = set(info['Chinese_name'])
chinese_names

{'一枝黄花',
 '丁公藤',
 '丁香',
 '三七',
 '三七花',
 '三分三',
 '三棱',
 '三白草',
 '三颗针',
 '丝瓜络',
 '两头尖',
 '两面针',
 '丹参',
 '乌敛莓',
 '乌梅',
 '乌梢蛇',
 '乌药',
 '九里香',
 '九香虫',
 '乳香',
 '了哥王根',
 '云芝',
 '五倍子',
 '五加皮',
 '五味子',
 '亚乎奴（锡生藤）',
 '亚麻子',
 '京大戟',
 '人参',
 '人参叶',
 '人工牛黄',
 '仙桃草',
 '仙茅',
 '仙鹤草',
 '伊贝母',
 '伸筋草',
 '体外培育牛黄',
 '何首乌',
 '余甘子',
 '佛手',
 '佩兰',
 '使君子',
 '侧柏叶',
 '倒扣草',
 '僵蚕',
 '儿茶',
 '党参',
 '全蝎',
 '八角茴香',
 '八角莲',
 '关黄柏',
 '冬凌草',
 '冬瓜子',
 '冬瓜皮',
 '冬葵果',
 '冬虫夏草',
 '冰片',
 '决明子',
 '凌霄花',
 '凤仙透骨草',
 '凤尾草',
 '凤眼草',
 '刀豆',
 '制何首乌',
 '制天南星',
 '制川乌',
 '制草乌',
 '刺五加',
 '前胡',
 '功劳木',
 '化橘红',
 '北刘寄奴',
 '北沙参',
 '北豆根',
 '十大功劳叶',
 '十大功劳根',
 '千年健',
 '千斤拔',
 '千日红',
 '千里光',
 '千金子',
 '千金子霜',
 '升麻',
 '半夏',
 '半枝莲',
 '半边莲',
 '华山参',
 '南五味子',
 '南板蓝根',
 '南沙参',
 '南瓜子',
 '南蛇藤果',
 '南鹤虱',
 '卷柏',
 '厚朴',
 '厚朴花',
 '合欢皮',
 '合欢花',
 '吴茱萸',
 '哈蟆油',
 '商陆',
 '四季青',
 '土木香',
 '土茯苓',
 '土荆皮',
 '土贝母',
 '土鳖虫（䗪虫）',
 '地枫皮',
 '地榆',
 '地耳草',
 '地肤子',
 '地锦草',
 '地骨皮',
 '地黄',
 '地龙',
 '垂盆草',
 '墓头回',
 '墨旱莲',
 '夏天无',
 '夏枯草',
 '大叶紫珠',
 '大枣',
 '大皂角',
 '大腹皮',
 '大蒜'

In [2]:
df = pd.read_excel("./data_update.xlsx")
df

,就诊年龄,EMPI,szy自编序号,开方时间,中医药方,味数,剂数,频次,途径,开立医生,Unnamed: 10,Unnamed: 11,≥3诊的处方
0,54.0,16228.0,1.0,2014-02-11 17:18:00,盐山萸肉（制山萸肉、盐/制山茱萸）10g 熟地黄（熟地）15g 山药(淮山)15g 茯苓(云...,12.0,28.0,NaN,水煎内服,刘旭生,NaN,NaN,NaN
1,54.0,16228.0,1.0,2014-03-04 16:38:00,盐山萸肉（制山萸肉、盐/制山茱萸）10g 熟地黄（熟地）15g 山药(淮山)15g 茯苓(云...,12.0,28.0,NaN,水煎内服,刘旭生,NaN,NaN,NaN
2,54.0,16228.0,1.0,2014-04-01 19:20:00,盐山萸肉（制山萸肉、盐/制山茱萸）10g 知母15g 山药(淮山)15g 茯苓(云苓)15g...,13.0,22.0,NaN,水煎内服,刘旭生,NaN,NaN,NaN
3,54.0,16228.0,1.0,2014-04-29 17:51:00,盐山萸肉（制山萸肉、盐/制山茱萸）10g 山药(淮山)15g 茯苓(云苓)15g 牡丹皮10...,12.0,26.0,NaN,水煎内服,刘旭生,NaN,NaN,NaN
4,54.0,16228.0,1.0,2014-05-27 19:46:00,盐山萸肉（制山萸肉、盐/制山茱萸）10g 山药(淮山)15g 茯苓(云苓)15g 牡丹皮10...,13.0,28.0,NaN,水煎内服,刘旭生,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
602,28.0,2028375.0,33.0,2014-05-20 16:13:00,菟丝子（盐菟丝子）15g 金樱子（盐金樱子）15g 甘草（甘草粒）5g 土茯苓20g 桃仁（...,14.0,7.0,NaN,水煎内服,刘旭生,NaN,NaN,NaN
603,28.0,2028375.0,33.0,2014-06-10 16:06:00,菟丝子（盐菟丝子）15g 甘草（甘草粒）5g 桃仁（?桃仁）10g 黄芪(北芪)30g 首乌...,13.0,14.0,NaN,水煎内服,刘旭生,NaN,NaN,NaN
604,28.0,2028375.0,33.0,2014-08-12 18:49:00,黄芪(北芪)30g 生地黄(生地)15g 制何首乌15g 桃仁（?桃仁）10g 盐山萸肉（制...,12.0,7.0,NaN,水煎内服,刘旭生,NaN,NaN,NaN
605,28.0,2028375.0,33.0,2014-09-09 14:53:00,黄芪(北芪)30g 生地黄(生地)15g 首乌藤（夜交藤）20g 桃仁（?桃仁）10g 盐山...,12.0,7.0,NaN,水煎内服,刘旭生,NaN,NaN,NaN


In [22]:
df = df.dropna(subset=['szy自编序号'])

In [23]:
df["szy自编序号"] = df["szy自编序号"].astype(int)
patients = df["szy自编序号"].unique()
patients

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33])

In [27]:
rename_map = {
    "广东王不留行": "王不留行",
    "炒薏苡仁":"薏苡仁"
}   

In [58]:
total = []
for n in patients:
    patient1 = df[df["szy自编序号"] == 1]
    data = patient1
    res_patient = []
    for idx, row in data.iterrows():
        res ={"visit": idx + 1}
        visit = idx + 1
        text = row["中医药方"]
        text_lst = text.split("g")
        for t in text_lst:
            t = t.strip().replace("（","(").replace("）",")")
            if t:
                # 使用正则表达式提取数字和名称
                match = re.search(r'(\d+)', t)
                if match:
                    num = int(match.group(1))
                    name = t[:match.start()].strip()
                    res[name] = num
        res_patient.append(res)
    total.append(res_patient)
total

[[{'visit': 1,
   '盐山萸肉(制山萸肉、盐/制山茱萸)': 10,
   '熟地黄(熟地)': 15,
   '山药(淮山)': 15,
   '茯苓(云苓)': 15,
   '牡丹皮': 10,
   '枳壳(炒枳壳)': 10,
   '佛手(制佛手)': 15,
   '茵陈(绵茵陈)': 15,
   '杜仲(盐杜仲)': 15,
   '桑寄生(杂寄生)': 15,
   '延胡索(醋延胡索/玄胡索/元胡/制延胡索)': 15,
   '炙甘草': 5},
  {'visit': 2,
   '盐山萸肉(制山萸肉、盐/制山茱萸)': 10,
   '熟地黄(熟地)': 15,
   '山药(淮山)': 15,
   '茯苓(云苓)': 15,
   '牡丹皮': 10,
   '杜仲(盐杜仲)': 15,
   '桑寄生(杂寄生)': 15,
   '延胡索(醋延胡索/玄胡索/元胡/制延胡索)': 15,
   '炙甘草': 5,
   '白芍': 15,
   '佛手(制佛手)': 15,
   '黄精(酒黄精)': 15},
  {'visit': 3,
   '盐山萸肉(制山萸肉、盐/制山茱萸)': 10,
   '知母': 15,
   '山药(淮山)': 15,
   '茯苓(云苓)': 15,
   '牡丹皮': 10,
   '黄精(酒黄精)': 15,
   '杜仲(盐杜仲)': 15,
   '桑寄生(杂寄生)': 15,
   '延胡索(醋延胡索/玄胡索/元胡/制延胡索)': 15,
   '甘草(甘草粒)': 5,
   '佛手(制佛手)': 15,
   '布渣叶': 15,
   '络石藤(广东络石藤)': 15},
  {'visit': 4,
   '盐山萸肉(制山萸肉、盐/制山茱萸)': 10,
   '山药(淮山)': 15,
   '茯苓(云苓)': 15,
   '牡丹皮': 10,
   '杜仲(盐杜仲)': 15,
   '桑寄生(杂寄生)': 15,
   '莲子(赤莲)': 15,
   '甘草(甘草粒)': 5,
   '芡实(茨实)': 15,
   '布渣叶': 15,
   '丹参': 15,
   '浙贝母': 15},
  {'visit': 5,
   '盐山萸肉(制山萸肉、盐

In [59]:
with open("data_update.json", "w") as f:
    json.dump(total, f, ensure_ascii=False, indent=2)

In [13]:
# 将index改为visit列，保存在newdata目录下
new_df.to_excel("./newdata/patient1_herbcnt.xlsx", index=False)